In [14]:
import pandas as pd
import numpy as np
import httpx
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
import os
import re
from tqdm import tqdm

In [15]:
load_dotenv(dotenv_path='../.env')

API_KEY = os.getenv("OPENAI_API_KEY")

In [16]:
def extract_rating(s: str) -> int:
    try:
        return int(re.findall(r'\d+', s)[0])
    except IndexError:
        return -1

In [20]:
import re


def query_gpt35(chiefcomplaint):

    header = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json',
    }

    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "system",
                "content": "You are an emergency department doctor.",
            },
            {
                "role": "user",
                "content": f"<s>[INST] You are an emergency department doctor. Please rate the following chief complaint on a scale of 1 to 10, where 10 is most severe. Provide only the rating in your answer. Start your answer with the rating in a numerical format. Like this: [RATING] [Explanation]. For example: 8. The chief complaint is very severe... Here is the chief complaint: {chiefcomplaint} [/INST]",
            },
        ],
        "max_tokens": 3,
    }

    url = 'https://api.openai.com/v1/chat/completions'

    try:
        response = httpx.post(url, headers=header, json=payload, timeout=30.0)
        response.raise_for_status()
        rating = response.json()['choices'][0]['message']['content']
    except (httpx.HTTPError, KeyError, IndexError) as e:
        print(f"An error occurred: {e}")
        rating = "None"

    return extract_rating(rating)


In [21]:
gpt4rated_frame = pd.read_csv('../processed_data/gpt4rated_frame.csv') #pd.read_csv('../processed_data/gpt4rated_frame.csv')

In [22]:
from time import sleep

# ...

# Create a ThreadPoolExecutor with max_workers=5
with ThreadPoolExecutor(max_workers=100) as executor:
    futures = []
    for i, row in tqdm(gpt4rated_frame.iterrows(), total=gpt4rated_frame.shape[0]):
        if row['rating'] == -1:
            # Submit each query_gpt35 call as a separate task to the executor
            future = executor.submit(query_gpt35, row['chiefcomplaint'])
            futures.append(future)

            if len(futures) == 100:
                # Wait for the first 5 tasks to complete before proceeding
                for completed_future in as_completed(futures):
                    rating = completed_future.result()
                    gpt4rated_frame.at[i, 'rating'] = rating
                futures = []

                gpt4rated_frame.to_csv('../processed_data/gpt4rated_frame.csv', index=False)
                sleep(1)

    # Wait for the remaining tasks to complete
    for completed_future in as_completed(futures):
        rating = completed_future.result()
        gpt4rated_frame.at[i, 'rating'] = rating

    gpt4rated_frame.to_csv('../processed_data/gpt4rated_frame.csv', index=False)


  9%|▉         | 5559/60407 [10:19<1:03:03, 14.50it/s]

An error occurred: Server error '502 Bad Gateway' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502


 26%|██▌       | 15708/60407 [30:04<1:38:11,  7.59it/s]

An error occurred: Server error '502 Bad Gateway' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502


 43%|████▎     | 26162/60407 [50:03<1:00:29,  9.44it/s]

...

 84%|████████▍ | 50708/60407 [1:35:56<15:19, 10.54it/s]

An error occurred: Server error '502 Bad Gateway' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502


 92%|█████████▏| 55608/60407 [1:44:48<11:39,  6.86it/s]

An error occurred: Server error '502 Bad Gateway' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502
An error occurred: Server error '502 Bad Gateway' for url 'https://api.openai.com/v1/chat/completions'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/502


100%|██████████| 60407/60407 [1:53:51<00:00,  8.84it/s]


In [24]:
gpt4rated_frame.to_csv('../processed_data/gpt4rated_frame.csv', index=False)

In [23]:
gpt4rated_frame["rating"].value_counts()

rating
-1     59803
 7       165
 10       91
 6        76
 2        72
 9        71
 5        66
 3        27
 4        25
 1        11
Name: count, dtype: int64